In [13]:
import pandas as pd 

In [14]:
data = pd.read_csv('train.csv')
data.drop('Id',axis=1 , inplace=True)
data

,Comment,Topic
0,A few things. You might have negative- frequen...,Biology
1,Is it so hard to believe that there exist part...,Physics
2,There are bees,Biology
3,I'm a medication technician. And that's alot o...,Biology
4,Cesium is such a pretty metal.,Chemistry
...,...,...
8690,I make similar observations over the last week...,Biology
8691,You would know.,Biology
8692,Also use the correct number of sig figs,Chemistry
8693,"What about the ethical delimmas, groundbreaki...",Biology


In [15]:
data.shape 
data.dtypes

Comment    object
Topic      object
dtype: object

In [16]:
CommentsToBeTokenized = data['Comment'] 
CommentsToBeTokenized

0       A few things. You might have negative- frequen...
1       Is it so hard to believe that there exist part...
2                                          There are bees
3       I'm a medication technician. And that's alot o...
4                          Cesium is such a pretty metal.
                              ...                        
8690    I make similar observations over the last week...
8691                                      You would know.
8692              Also use the correct number of sig figs
8693    What about the ethical delimmas,  groundbreaki...
8694                            I would like to know too.
Name: Comment, Length: 8695, dtype: object

In [17]:
tokens = CommentsToBeTokenized[0].split()
tokens

['A',
 'few',
 'things.',
 'You',
 'might',
 'have',
 'negative-',
 'frequency',
 'dependent',
 'selection',
 'going',
 'on',
 'where',
 'the',
 'least',
 'common',
 'phenotype,',
 'reflected',
 'by',
 'genotype,',
 'is',
 'going',
 'to',
 'have',
 'an',
 'advantage',
 'in',
 'the',
 'environment.',
 'For',
 'instance,',
 'if',
 'a',
 'prey',
 'animal',
 'such',
 'as',
 'a',
 'vole',
 'were',
 'to',
 'have',
 'a',
 'light',
 'and',
 'a',
 'dark',
 'phenotype,',
 'a',
 'predator',
 'might',
 'recognize',
 'the',
 'more',
 'common',
 'phenotype',
 'as',
 'food.',
 'So',
 'if',
 'the',
 'light',
 'voles',
 'are',
 'more',
 'common,',
 'foxes',
 'may',
 'be',
 'keeping',
 'a',
 'closer',
 'eye',
 'out',
 'for',
 'light',
 'phenotypic',
 'voles,',
 'recognising',
 'them',
 'as',
 'good',
 'prey.',
 'This',
 'would',
 'reduce',
 'the',
 'light',
 'causing',
 'alleles',
 'due',
 'to',
 'increased',
 'predation',
 'and',
 'the',
 'dark',
 'genotypes',
 'would',
 'increase',
 'their',
 'proport

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform(CommentsToBeTokenized)

In [19]:
bow

<8695x18177 sparse matrix of type '<class 'numpy.int64'>'
	with 190076 stored elements in Compressed Sparse Row format>

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()

In [21]:
tfidf_result.shape

(8695, 18177)

In [22]:
vectorizer = TfidfVectorizer(max_features=18177)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
NB= MultinomialNB()

In [24]:
xtrain, xtest, ytrain, ytest = train_test_split(tfidf_result, data['Topic'], test_size= 0.1)
NB.fit(xtrain, ytrain)

MultinomialNB()

In [25]:
ypred1 = NB.predict(xtrain)
ypred = NB.predict(xtest)

In [26]:
ypred.shape

(870,)

In [27]:
ytrain=ytrain[0:870]
ytest=ytest[0:870]

In [28]:
print("Training Results:\n")
print(classification_report(ytrain, ypred1))
print("\nTesting Results:\n")
print(classification_report(ytest, ypred))

Training Results:



ValueError: Found input variables with inconsistent numbers of samples: [870, 7825]

In [29]:
df = data.groupby('Topic')

In [30]:
df.count()

,Comment
Topic,
Biology,3591
Chemistry,2920
Physics,2184


In [31]:
import sys # allows for command line arguments to be used as arguments for script.

# the 0th argument (shoud be in ""s) is the question
question = sys.argv[0] 
# the 1th argument ("Physics", "Chemistry", or "Biology") is the correct label to be checked with 
right_Label = sys.argv[1] 

In [51]:
def online_test(question, right_Label):
    # import the data
    dataList = {"Comment": [question], "Right_Topic": [right_Label], "Guess_Topic": [""]}
    data = pd.DataFrame(dataList)
    print(data)

    # tokenize the question
    tokens = question.split()

    # vectorize the question
    vectorizer = TfidfVectorizer(max_features=6000)
    tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
    features = vectorizer.get_feature_names_out()
    tfidf_result = pd.DataFrame(tfidf_result, columns=features)
    
    # predict the label
    prediction = NB.predict(tfidf_result)
    data["Guess_Topic"] = prediction
    print(data)
    
    # check if the prediction is correct
    if prediction == right_Label:
        return "Correct"
    else:
        return "Incorrect"

In [52]:
online_test("In another example, STP will be associated with 1 atm and 273 K. For part (b), the ideal gas equation (PV = nRT) and the solution (~22.4 L/mol) will be given to the machine. Based on this question and similar questions which may ask for other" , "Chemistry")

                                             Comment Right_Topic Guess_Topic
0  In another example, STP will be associated wit...   Chemistry            


c:\Users\Richie\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- 00
- 00000000000000000000000332
- 00004
- 00047
- 000705
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 6000 features, but MultinomialNB is expecting 18177 features as input.

: 

: 

: 

: 